In [1]:
%pip install -r ~/code/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/UPennContrast/devops/girder/annotation_client/
%pip install pandas
%pip install imageio
%pip install rtree
%pip install scikit-image
%pip install shapely
%pip install nd2reader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the

In [2]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
import annotation_tools
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np
import math

/Users/arjunraj/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [4]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [5]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,DDX58_AXL_EGFR_well2
1,GGTEST
2,normmedia_8well_col2_livecellgfp


In [6]:
dataset_name = 'normmedia_8well_col2_livecellgfp'

In [7]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
print(datasetClient.tiles)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

{'IndexRange': {'IndexC': 2, 'IndexT': 4, 'IndexXY': 6, 'IndexZ': 7}, 'IndexStride': {'IndexC': 1, 'IndexT': 14, 'IndexXY': 56, 'IndexZ': 2}, 'bandCount': 1, 'channelmap': {'Brightfield': 0, 'YFP': 1}, 'channels': ['Brightfield', 'YFP'], 'dtype': 'uint16', 'frames': [{'Channel': 'Brightfield', 'Frame': 0, 'Index': 0, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'YFP', 'Frame': 1, 'Index': 0, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'Brightfield', 'Frame': 2, 'Index': 1, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'YFP', 'Frame': 3, 'Index': 1, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'Brightfield', 'Frame': 4, 'Index': 2, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'YFP', 'Frame': 5, 'Index': 2, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'Brightfield', 'Frame': 6, 'Index': 3, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 3}, {'Channel': 'YFP', 'F

In [11]:
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='polygon')
lineAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='line')

In [14]:
print(len(blobAnnotationList))
pprint.pprint(blobAnnotationList)

10
[{'_id': '656e37f8b98181709b141186',
  'access': {'groups': [],
             'users': [{'flags': [],
                        'id': '65689787d45237881913b68f',
                        'level': 2}]},
  'channel': 1,
  'coordinates': [{'x': 258.64339697093936, 'y': 391.3640923301282},
                  {'x': 258.64339697093936, 'y': 398.7167101708975},
                  {'x': 258.23491820200775, 'y': 401.5760615534189},
                  {'x': 256.1925243573496, 'y': 402.39301909128216},
                  {'x': 245.57207636512726, 'y': 401.1675827844873},
                  {'x': 240.6703311379477, 'y': 399.9421464776924},
                  {'x': 238.62793729328956, 'y': 398.7167101708975},
                  {'x': 236.99402221756304, 'y': 397.08279509517104},
                  {'x': 235.36010714183652, 'y': 394.2234437126496},
                  {'x': 234.9516283729049, 'y': 391.77257109905986},
                  {'x': 235.36010714183652, 'y': 389.32169848547005},
                  {'x':

In [9]:
import pprint
print(datasetId)
connectionList = annotationClient.getAnnotationConnections(datasetId)
pprint.pprint(connectionList,indent=2)

656e377ab98181709b14106c
[ { '_id': '656e3840b98181709b14153d',
    'access': { 'groups': [],
                'users': [ { 'flags': [],
                             'id': '65689787d45237881913b68f',
                             'level': 2}]},
    'childId': '656e3806b98181709b141231',
    'datasetId': '656e377ab98181709b14106c',
    'parentId': '656e3802b98181709b141201',
    'tags': ['nucleus']},
  { '_id': '656e3840b98181709b14153e',
    'access': { 'groups': [],
                'users': [ { 'flags': [],
                             'id': '65689787d45237881913b68f',
                             'level': 2}]},
    'childId': '656e3815b98181709b1412fa',
    'datasetId': '656e377ab98181709b14106c',
    'parentId': '656e3813b98181709b1412d9',
    'tags': ['nucleus']},
  { '_id': '656e3840b98181709b14153f',
    'access': { 'groups': [],
                'users': [ { 'flags': [],
                             'id': '65689787d45237881913b68f',
                             'level': 2}]},
    '

In [24]:
# Code below is for finding the parent/child annotations of a given tag.
# Used for making a worker to return parent and child ID as a property of the annotation.
exclusive = False
tags = set(['nucleus'])
annotationConnectionList = {}  # Initialize as a dictionary
for connection in connectionList:
    child_tags = set(annotationClient.getAnnotationById(connection['childId'])['tags'])
    
    # if the child matches the tags, then add the parent to the list as a parent of the current child annotation.
    if (exclusive and (child_tags == tags)) or ((not exclusive) and (len(child_tags & tags) > 0)):
        if connection['childId'] not in annotationConnectionList:
            annotationConnectionList[connection['childId']] = { 'parentId': connection['parentId'], 'childId': None }
        else:
            annotationConnectionList[connection['childId']]['parentId'] = connection['parentId']

    parent_tags = set(annotationClient.getAnnotationById(connection['parentId'])['tags'])
    
    # if the parent matches the tags, then add the child to the list as a child of the current parent annotation.
    if (exclusive and (parent_tags == tags)) or ((not exclusive) and (len(parent_tags & tags) > 0)):
        if connection['parentId'] not in annotationConnectionList:
            annotationConnectionList[connection['parentId']] = { 'parentId': None, 'childId': connection['childId'] }
        else:
            annotationConnectionList[connection['parentId']]['childId'] = connection['childId']


pprint.pprint(annotationConnectionList)

{'656e37f8b98181709b141186': {'childId': '656e37ffb98181709b1411db',
                              'parentId': None},
 '656e37ffb98181709b1411db': {'childId': '656e3802b98181709b141201',
                              'parentId': '656e37f8b98181709b141186'},
 '656e3802b98181709b141201': {'childId': '656e3806b98181709b141231',
                              'parentId': '656e37ffb98181709b1411db'},
 '656e3806b98181709b141231': {'childId': None,
                              'parentId': '656e3802b98181709b141201'},
 '656e380db98181709b14128e': {'childId': '656e3810b98181709b1412b8',
                              'parentId': None},
 '656e3810b98181709b1412b8': {'childId': '656e3813b98181709b1412d9',
                              'parentId': '656e380db98181709b14128e'},
 '656e3813b98181709b1412d9': {'childId': '656e3815b98181709b1412fa',
                              'parentId': '656e3810b98181709b1412b8'},
 '656e3815b98181709b1412fa': {'childId': None,
                              'parentId

In [25]:
id_to_integer_mapping = {}
current_integer = 0

for connection in connectionList:
    for key in ['childId', 'parentId']:
        id_value = connection[key]
        if id_value not in id_to_integer_mapping:
            id_to_integer_mapping[id_value] = current_integer
            current_integer += 1

pprint.pprint(id_to_integer_mapping)

{'656e37f8b98181709b141186': 6,
 '656e37ffb98181709b1411db': 4,
 '656e3802b98181709b141201': 1,
 '656e3806b98181709b141231': 0,
 '656e380db98181709b14128e': 7,
 '656e3810b98181709b1412b8': 5,
 '656e3813b98181709b1412d9': 3,
 '656e3815b98181709b1412fa': 2}


In [15]:
def find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=True):
    params = {'Time': Time, 'XY': XY, 'Z': Z}
    return [target for target in target_list if all(source['location'].get(attr) == target['location'].get(attr) for attr, value in params.items() if value)]

def find_closest_point(source, target_list):
    min_distance = float('inf')
    closest_point = None
    for target in target_list:
        if source['_id'] == target['_id']:
            continue
        distance = calculate_distance(source, target)
        if distance < min_distance:
            min_distance = distance
            closest_point = target
    return closest_point, min_distance

In [19]:
inTags = ['outerPt']
filteredPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,inTags,exclusive=False)
print(len(filteredPointList))

# workerInterface = params['workerInterface']
tags = ['innerPt'] # set(workerInterface.get('Tags of points to measure distance to', None))
exclusive = False # workerInterface['Target tag match'] == 'Exact'
#targetPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,tags,exclusive=exclusive)
#print(len(targetPointList))

9
10


In [26]:
myPoint = filteredPointList[7]
children = annotationClient.getAnnotationConnections(datasetId,parentId=myPoint['_id'])
print(len(children))
pprint.pprint(children[0])


1
{'_id': '64aa9ac3f69bc320e7503878',
 'access': {'groups': [],
            'users': [{'flags': [],
                       'id': '62891fc02060d362984bc104',
                       'level': 2}]},
 'childId': '64aa9aa1f69bc320e7503870',
 'datasetId': '64aa2973f69bc320e7503648',
 'label': 'Connection Add outerPt to innerPt',
 'parentId': '64aa9aa7f69bc320e7503876',
 'tags': ['outerPt', 'innerPt']}


In [34]:
def getChildAnnotationsFromConnections(connections):
    child_annotations = []
    for connection in connections:
        childId = connection.get('childId')
        if childId:
            child_annotation = annotationClient.getAnnotationById(childId)
            child_annotations.append(child_annotation)
    return child_annotations

In [35]:
def calculate_distance(point1, point2):
    x1, y1, z1 = point1['coordinates'][0]['x'], point1['coordinates'][0]['y'], point1['coordinates'][0]['z']
    x2, y2, z2 = point2['coordinates'][0]['x'], point2['coordinates'][0]['y'], point2['coordinates'][0]['z']
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)


In [36]:
pprint.pprint(getChildAnnotationsFromConnections(children))

[{'_id': '64aa9aa1f69bc320e7503870',
  'access': {'groups': [],
             'users': [{'flags': [],
                        'id': '62891fc02060d362984bc104',
                        'level': 2}]},
  'channel': 1,
  'coordinates': [{'x': 394.80699568892766, 'y': 399.6508790623366, 'z': 0}],
  'datasetId': '64aa2973f69bc320e7503648',
  'location': {'Time': 0, 'XY': 0, 'Z': 0},
  'shape': 'point',
  'tags': ['innerPt']}]


In [38]:
for source in filteredPointList:
        connections = annotationClient.getAnnotationConnections(datasetId,parentId=source['_id'])
        targetPointList = getChildAnnotationsFromConnections(connections)
        
        # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
        closest_point, min_distance = find_closest_point(source, targetPointList)
        
        if min_distance != float('inf'):
            print(min_distance) #workerClient.add_annotation_property_values(source, min_distance)

7.65561016396188
10.806361218119244


BELOW HERE IS ALL JUST OLD STUFF

In [35]:
point1 = pointAnnotationList[0]
#pprint.pprint(point1)
myPoint = {'channel': 0,'coordinates': [{'x': 600.8816931197485, 'y': 430.72920634951333, 'z': 0}],
 'datasetId': '64aa2973f69bc320e7503648',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'point',
 'tags': ['outerPt']}

#pprint.pprint(myPoint)
annotationClient.createAnnotation(myPoint)

{'_id': '64b55ca5409e03c7075a87c3',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
 'channel': 0,
 'coordinates': [{'x': 600.8816931197485, 'y': 430.72920634951333, 'z': 0}],
 'datasetId': '64aa2973f69bc320e7503648',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'point',
 'tags': ['outerPt']}

In [9]:
filteredPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,['outerPt'],exclusive=False)
print(len(filteredPointList))
targetPointList = annotation_tools.get_annotations_with_tags(pointAnnotationList,['innerPt'],exclusive=False)
print(len(targetPointList))

8
10


In [31]:
annotation1 = filteredPointList[0].copy()
annotation2 = targetPointList[0].copy()
print(datasetId)
pprint.pprint(annotation1)
pprint.pprint(annotation2)

combined_tags = list(set(annotation1['tags'] + annotation2['tags']))
print(combined_tags)

myNewConnection = {'datasetId': datasetId,'parentId': annotation1['_id'], 'childId': annotation2['_id'],'tags': combined_tags}
pprint.pprint(myNewConnection)

#annotationClient.client.post(PATHS['connection'], json=myNewConnection)
annotationClient.createConnection(myNewConnection)


64aa2973f69bc320e7503648
{'_id': '64aa9970f69bc320e7503830',
 'access': {'groups': [],
            'users': [{'flags': [],
                       'id': '62891fc02060d362984bc104',
                       'level': 2}]},
 'channel': 0,
 'coordinates': [{'x': 288.8816931197485, 'y': 430.72920634951333, 'z': 0}],
 'datasetId': '64aa2973f69bc320e7503648',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'point',
 'tags': ['outerPt']}
{'_id': '64aa99e3f69bc320e7503856',
 'access': {'groups': [],
            'users': [{'flags': [],
                       'id': '62891fc02060d362984bc104',
                       'level': 2}]},
 'channel': 1,
 'coordinates': [{'x': 323.7430687831708, 'y': 462.975978838179, 'z': 0}],
 'datasetId': '64aa2973f69bc320e7503648',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'point',
 'tags': ['innerPt']}
['outerPt', 'innerPt']
{'childId': '64aa99e3f69bc320e7503856',
 'datasetId': '64aa2973f69bc320e7503648',
 'parentId': '64aa9970f69bc320e7503830',
 'tags': 

HttpError: HTTP error 400: POST http://localhost:8080/api/v1//annotation_connection/
Response text: {"message": "'tags' is a required property\n\nFailed validating 'required' in schema:\n    {'$schema': 'http://json-schema.org/schema#',\n     'id': '/girder/plugins/upenncontrast_annotation/models/annotation_connection',\n     'properties': {'childId': {'type': 'string'},\n                    'datasetId': {'type': 'string'},\n                    'label': {'type': 'string'},\n                    'parentId': {'type': 'string'},\n                    'tags': {'items': {'type': 'string'}, 'type': 'array'}},\n     'required': ['parentId', 'childId', 'datasetId', 'tags'],\n     'type': 'object'}\n\nOn instance:\n    {'access': {'groups': [],\n                'users': [{'flags': [],\n                           'id': ObjectId('62891fc02060d362984bc104'),\n                           'level': 2}]},\n     'childId': '64aa99e3f69bc320e7503856',\n     'datasetId': '64aa2973f69bc320e7503648',\n     'p

In [30]:
data = {
    'access': {
        'groups': [],
        'users': [
            {
                'flags': [],
                'id': '62891fc02060d362984bc104',
                'level': 2
            }
        ]
    },
    'childId': '64aa99e3f69bc320e7503856',
    'datasetId': '64aa2973f69bc320e7503648',
    'parentId': '64aa9970f69bc320e7503830',
    "tags": ["innerPt", "outerPt"]  # Add this line
}

annotationClient.createConnection(data)

HttpError: HTTP error 400: POST http://localhost:8080/api/v1//annotation_connection/
Response text: {"message": "'tags' is a required property\n\nFailed validating 'required' in schema:\n    {'$schema': 'http://json-schema.org/schema#',\n     'id': '/girder/plugins/upenncontrast_annotation/models/annotation_connection',\n     'properties': {'childId': {'type': 'string'},\n                    'datasetId': {'type': 'string'},\n                    'label': {'type': 'string'},\n                    'parentId': {'type': 'string'},\n                    'tags': {'items': {'type': 'string'}, 'type': 'array'}},\n     'required': ['parentId', 'childId', 'datasetId', 'tags'],\n     'type': 'object'}\n\nOn instance:\n    {'access': {'groups': [],\n                'users': [{'flags': [],\n                           'id': ObjectId('62891fc02060d362984bc104'),\n                           'level': 2}]},\n     'childId': '64aa99e3f69bc320e7503856',\n     'datasetId': '64aa2973f69bc320e7503648',\n     'p

In [10]:
import math

def calculate_distance(point1, point2):
    x1, y1, z1 = point1['coordinates'][0]['x'], point1['coordinates'][0]['y'], point1['coordinates'][0]['z']
    x2, y2, z2 = point2['coordinates'][0]['x'], point2['coordinates'][0]['y'], point2['coordinates'][0]['z']
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)


def find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=True):
    """
    This function filters the target_list based on the 'location' of the source point.
    The function parameters 'Time', 'XY', and 'Z' can be set to True or False to specify whether these 'location' attributes need to be matched.
    By default, all of these parameters are set to True, meaning all 'location' attributes need to match.

    Parameters:
    source (dict): The source point annotation object
    target_list (list): The list of target point annotation objects
    Time (bool): Specifies whether the 'Time' attribute of 'location' needs to be matched. Default is True.
    XY (bool): Specifies whether the 'XY' attribute of 'location' needs to be matched. Default is True.
    Z (bool): Specifies whether the 'Z' attribute of 'location' needs to be matched. Default is True.

    Returns:
    list: The filtered list of target point annotation objects that match the specified 'location' attributes

    Example of usage:
    1) Matching all 'location' attributes:
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list)

    2) Matching specified 'location' attributes (in this case, 'Time' and 'XY'):
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=False)
    """
    params = {'Time': Time, 'XY': XY, 'Z': Z}
    return [target for target in target_list if all(source['location'].get(attr) == target['location'].get(attr) for attr, value in params.items() if value)]

def find_closest_point(source, target_list):
    min_distance = float('inf')
    closest_point = None
    for target in target_list:
        if source['_id'] == target['_id']:
            continue
        distance = calculate_distance(source, target)
        if distance < min_distance:
            min_distance = distance
            closest_point = target
    return closest_point, min_distance

#filteredPointList = [...]  # Your filteredPointList here
#targetPointList = [...]  # Your targetPointList here

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList, Time=True, XY=True)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")



For point id 64aa9970f69bc320e7503830, closest point is 64aa99e4f69bc320e7503858 with distance 44.97019871692391
For point id 64aa9971f69bc320e7503832, closest point is 64aa99e4f69bc320e7503858 with distance 40.542745732502006
For point id 64aa9972f69bc320e7503834, closest point is 64aa99ecf69bc320e7503864 with distance 22.147569722466812
For point id 64aa9972f69bc320e7503836, closest point is 64aa99e3f69bc320e7503856 with distance 23.26450919111039
For point id 64aa9973f69bc320e7503838, closest point is 64aa99e4f69bc320e7503858 with distance 15.882769465319464
For point id 64aa9974f69bc320e750383a, closest point is 64aa99e4f69bc320e7503858 with distance 36.323228159944755
For point id 64aa9990f69bc320e7503852, closest point is 64aa9a46f69bc320e750386c with distance 7.65561016396188


TypeError: 'NoneType' object is not subscriptable

In [28]:
print(closest_point)
print(min_distance)

None
inf


In [25]:
import math

def calculate_distance(p1, p2):
    return math.sqrt((p2['x'] - p1['x'])**2 + (p2['y'] - p1['y'])**2 + (p2['z'] - p1['z'])**2)

def total_length(line):
    coordinates = line['coordinates']
    return sum(calculate_distance(coordinates[i], coordinates[i+1]) for i in range(len(coordinates)-1))

myLine = lineAnnotationList[2]
print(total_length(myLine))


164.89992701209385


Now let's try to push a new annotation

In [40]:
# Make a small square at the top left of the image
square_coords = [(10, 10),(30, 10),(30, 30),(10, 30)]

newAnnotation = {
    "tags": ['newTag'], # *** NEED TO UPDATE TO ADD A NEW TAG ****
    "shape": "polygon",
    "channel": blobAnnotationList[0]['channel'],
    "location": blobAnnotationList[0]['location'],
    "datasetId": blobAnnotationList[0]['datasetId'],
    "coordinates": [{"x": float(coord[0]), "y": float(coord[1])} for coord in square_coords]
}
annotationClient.createAnnotation(newAnnotation)


{'_id': '64a7006f1ed2b05b5e1d8b83',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
 'channel': 1,
 'coordinates': [{'x': 10.0, 'y': 10.0},
  {'x': 30.0, 'y': 10.0},
  {'x': 30.0, 'y': 30.0},
  {'x': 10.0, 'y': 30.0}],
 'datasetId': '649b6efb5d422edecab7f9d8',
 'location': {'Time': 0, 'XY': 0, 'Z': 0},
 'shape': 'polygon',
 'tags': ['newTag']}